# Get Training Data

In [3]:
import os
import sys
from pathlib import Path # if you haven't already done so
file = Path(os.getcwd()).resolve()
parent, root = file.parent, file.parents[1]
print(parent, root)
sys.path.append(str(parent))
# sys.path.append("..")

# # Additionally remove the current file's directory from sys.path
# try:
#     sys.path.remove(str(parent))
# except ValueError: # Already removed
#     pass

/root/fruitao/drone/logic_guided_prompt/code /root/fruitao/drone/logic_guided_prompt


In [4]:
from logic.description_logic import *
from utils.load_sensor_data import *
from data_augmentation.text_augmentation import *

/root/fruitao/drone/logic_guided_prompt/code/data_augmentation /root/fruitao/drone/logic_guided_prompt/code


In [18]:
# old_format_example=pd.ExcelFile(r"../text_annotation/rural.xlsx")

dataset_file_li=[r"../../text_annotation/desert.xlsx", r"../../text_annotation/disturbance.xlsx", r"../../text_annotation/factory.xlsx", r"../../text_annotation/island.xlsx", 
                 r"../../text_annotation/misc.xlsx", r"../../text_annotation/ocean.xlsx", r"../../text_annotation/rural.xlsx", r"../../text_annotation/urban.xlsx"
]

# dataset_file_li=[ r"../text_annotation/rural.xlsx"]

old_data_dic=dict()

for dataset_file in dataset_file_li:
    print("----processing file: {}----".format(dataset_file))
    
    old_format_example=pd.ExcelFile(dataset_file)

    ts_converters={'name': str, 'Type': str, 'Moving': str2bool, 'InPath': str2bool, 'time_stamp': str}
    status_converters={'PilotExperienced': str2bool, 'Low_visibility': str2bool, 'Normal_frame': str2bool,
                      'weather': str, 'upside_down': str2bool, 'good_motor_condition': str2bool,
                      'going_backwards': str2bool, 'indoor': str2bool, 'waterproof drone?': str2bool,
                      'flying over': str, 'criticality': str, 
                      'RiskOfPhysicalDamage': str, 'RiskOfInternalDamage': str2bool,
                      'RiskOfHumanDamage': str2bool, 'LostConnection': str2bool}
    text_converters={'Link':str, 'Text1':str, 'Text2':str}

    old_format_ts=pd.read_excel(old_format_example, sheet_name='timestep', converters=ts_converters)
    old_format_status=pd.read_excel(old_format_example, sheet_name='status', converters=status_converters)
    old_format_text=pd.read_excel(old_format_example, sheet_name='text', converters=text_converters)

    print("---loading sensor data---")
    load_ts_to_dict(old_format_ts, old_data_dic)
    load_status_to_dict(old_format_status, old_data_dic)
    load_text_to_dict(old_format_text, old_data_dic)

    # print("---generating templates---")
    for link, data in old_data_dic.items():
        templates, related_status_dic, related_timestep_dic = gen_templates(data)
        old_data_dic[link]['templates'] = templates
        old_data_dic[link]['related_status_dic'] = related_status_dic
        old_data_dic[link]['related_timestep_dic'] = related_timestep_dic


----processing file: ../../text_annotation/desert.xlsx----
---loading sensor data---
templates:{'flying_not_normal_frame': ['The drone is flying with a damaged frame.'], 'near_moving_object': ["(0:01) The drone is risking physical damage. It's flying too close to the moving car at a distance of 3.0m.", "(0:05) The drone is risking physical damage. It's flying too close to the moving car at a distance of 0.3m."], 'reachable_inpath': ["(0:05) The drone is facing the risk of physical damage. There is a car in the drone's flight path at a distance of 0.3m."]}
templates:{'near_moving_object': ["(0:01) The drone is risking physical damage. It's flying too close to the moving car at a distance of 3.0m.", "(0:03) The drone is risking physical damage. It's flying too close to the moving car at a distance of 0.2m."], 'reachable_inpath': ["(0:03) The drone is facing the risk of physical damage. There is a car in the drone's flight path at a distance of 0.2m."]}
templates:{'flying_not_normal_frame

In [19]:
# print("old_data_dic['https://drive.google.com/file/d/1oJ6t-VJN3rAmslsXszkVUzTZl4LJujeM/view?usp=sharing']['timestep'][0]['time_stamp']:{}".format(old_data_dic['https://drive.google.com/file/d/1oJ6t-VJN3rAmslsXszkVUzTZl4LJujeM/view?usp=sharing']['timestep'][0]['time_stamp']))
print("len(old_data_dic):{}".format(len(old_data_dic)))
print("--['text'][0]:{}".format(
    old_data_dic['https://drive.google.com/file/d/1oJ6t-VJN3rAmslsXszkVUzTZl4LJujeM/view?usp=sharing']['text'][0]))
print("--['related_status_dic']:{}".format(
    old_data_dic['https://drive.google.com/file/d/1oJ6t-VJN3rAmslsXszkVUzTZl4LJujeM/view?usp=sharing']['related_status_dic']))
print("--['related_timestep_dic']:{}".format(
    old_data_dic['https://drive.google.com/file/d/1oJ6t-VJN3rAmslsXszkVUzTZl4LJujeM/view?usp=sharing']['related_timestep_dic']))
print("--['templates']:{}".format(
    old_data_dic['https://drive.google.com/file/d/1oJ6t-VJN3rAmslsXszkVUzTZl4LJujeM/view?usp=sharing']['templates']))

len(old_data_dic):300
--['text'][0]:{'Text1': "Risk of Physical Damage! The drone has a damaged frame and empty battery. It's flying at an altitude of 5m. [SEP]\n(0:01) Risk of Physical Damage! There are two gravestones in the drone's flight path at a distance of 3m.  [SEP] \n(0:05) Risk of Physical Damage! Now there's a castle in the drone's flight path at a distance of 1m. [SEP]", 'Text2': "Risk of physical damage! The drone is flying with an empty battery. The frame is also damaged and the altitude is 5m. [SEP]\n(0:01) Risk of physical damage! There are two gravestones 3m away and in the drone's path. [SEP]\n(0:05) Risk of physical damage! The drone is flying toward a castle in its path which is only 1m away. [SEP]"}
--['related_status_dic']:{'flying_empty_battery': [{'Battery_level': 0}], 'flying_not_normal_frame': [{'Normal_frame': False}]}
--['related_timestep_dic']:{'reachable_inpath': [{'time_stamp': '0:01', 'name': 'gravestone 1', 'Distance': 3.0}, {'time_stamp': '0:01', 'name

In [20]:
print(old_data_dic['https://drive.google.com/file/d/1oJ6t-VJN3rAmslsXszkVUzTZl4LJujeM/view?usp=sharing'].keys())

dict_keys(['timestep', 'status', 'text', 'templates', 'related_status_dic', 'related_timestep_dic'])


In [21]:
import json

link_li=[]
text_li=[]
dl_name_li=[]
templates_li=[]
related_sensor_data_li=[]
related_status_dic_li=[]
related_timestep_dic_li=[]
dl_related_sensor_data_mapping_li=[]

# templates_nested_li=[]
# related_status_dic_nested_li=[]
# related_timestep_dic_nested_li=[]
# related_sensor_data_nested_li=[]

# related_sensor_data_li as input and templates_li as label
for link, value in old_data_dic.items():
#     print("value:{}".format(value))
    link_li.append(link)
    if 'text' not in value:
        text_li.append([])
    else:    
        text_li.append(value['text'])
    dl_index=0
    
#     dl_related_sensor_data_mapping={}
#     related_status_dic_li=[]
#     related_timestep_dic_li=[]
#     related_sensor_data_li=[]
#     templates_li=[]
    for dl_name, template_value in value['templates'].items():
        templates_li.extend(template_value)
        if dl_name in value['related_status_dic']:
            related_status_dic_li.extend(value['related_status_dic'][dl_name])
            related_sensor_data_li.extend(value['related_status_dic'][dl_name])
        elif dl_name in value['related_timestep_dic']:
            related_timestep_dic_li.extend(value['related_timestep_dic'][dl_name])
            related_sensor_data_li.extend(value['related_timestep_dic'][dl_name])
        else:
            print("WHAT?")
        
related_sensor_data_li=[json.dumps(dic) for dic in related_sensor_data_li]
#         dl_related_sensor_data_mapping[dl_name]=[i+dl_index for i in range(len(template_value))]
#         dl_index+=len(template_value)
        
#     dl_related_sensor_data_mapping_li.append(dl_related_sensor_data_mapping)
#     related_status_dic_nested_li.append(related_status_dic_li)
#     related_timestep_dic_nested_li.append(related_timestep_dic_li)
#     templates_nested_li.append(templates_li)
#     related_sensor_data_nested_li.append(related_sensor_data_li)

In [22]:
import math

def train_test_split(all_li, train_ratio=0.6):
    all_len=len(all_li)
    train_len=math.ceil(train_ratio*all_len)
    
    return all_li[:train_len], all_li[train_len:]

train_sensor_data, test_sensor_data=train_test_split(related_sensor_data_li)
train_templates, test_templates=train_test_split(templates_li)

print(len(related_sensor_data_li), len(train_sensor_data), len(test_sensor_data))
print(len(templates_li), len(train_templates), len(test_templates))
    

1652 992 660
1652 992 660


In [23]:
# mapping{'flying_empty_battery':[0], 'flying_not_normal_frame':[1], 'reachable_inpath':[2, 3, 4]}
# input: [{'Battery_level': 0}, {'Normal_frame': False}, {'time_stamp': '0:01', 'name': 'gravestone 1', 'Distance': 3.0}, {'time_stamp': '0:01', 'name': 'gravestone 2', 'Distance': 3.0}, {'time_stamp': '0:05', 'name': 'castle ', 'Distance': 1.0}]
# output: 'The flying drone is runing out of battery with only 0% charge.'

print(len(related_sensor_data_li), related_sensor_data_li[0], type(related_sensor_data_li[0]))
print(len(templates_li), templates_li[0], type(templates_li[0]))
# print(dl_related_sensor_data_mapping_li[2])

1652 {"Normal_frame": false} <class 'str'>
1652 The drone is flying with a damaged frame. <class 'str'>


In [24]:
from torch.utils.data import Dataset

class drone_dataset(Dataset):
    def __init__(self, data_dic):
        self.data_dic=data_dic
        
    
    def __len__(self):
        return len(self.data_dic)
    
    def __getitem__(self, idx):
        return data_dic['related_status_dic'][0], data_dic['related_timestep_dic'], data_dic['text'][0]
    
    

# Augmented Prompt

In [25]:
!nvidia-smi

Fri Jun  9 15:03:00 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 515.86.01    Driver Version: 515.86.01    CUDA Version: 11.7     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A100 80G...  On   | 00000000:01:00.0 Off |                    0 |
| N/A   32C    P0    63W / 300W |  11691MiB / 81920MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
|   1  NVIDIA A100 80G...  On   | 00000000:22:00.0 Off |                    0 |
| N/A   

In [26]:
import torch

device=torch.device("cuda:3")

In [27]:
import math

from tqdm import tqdm
import numpy as np
from transformers import T5Tokenizer, T5ForConditionalGeneration
import torch.nn as nn
from sklearn.metrics import accuracy_score

In [28]:
class SoftEmbedding(nn.Module):
    
    def __init__(self, 
                wte: nn.Embedding,
                n_tokens: int = 10, 
                random_range: float = 0.5,
                initialize_from_vocab: bool = True):
        """appends learned embedding to 
        Args:
            wte (nn.Embedding): original transformer word embedding
            n_tokens (int, optional): number of tokens for task. Defaults to 10.
            random_range (float, optional): range to init embedding (if not initialize from vocab). Defaults to 0.5.
            initialize_from_vocab (bool, optional): initalizes from default vocab. Defaults to True.
        """
        super(SoftEmbedding, self).__init__()
        self.wte = wte
        self.n_tokens = n_tokens
        self.learned_embedding = nn.parameter.Parameter(self.initialize_embedding(wte,
                                                                               n_tokens, 
                                                                               random_range, 
                                                                               initialize_from_vocab))
            
    def initialize_embedding(self, 
                             wte: nn.Embedding,
                             n_tokens: int = 10, 
                             random_range: float = 0.5, 
                             initialize_from_vocab: bool = True):
        """initializes learned embedding
        Args:
            same as __init__
        Returns:
            torch.float: initialized using original schemes
        """
        if initialize_from_vocab:
            return self.wte.weight[:n_tokens].clone().detach()
        return torch.FloatTensor(n_tokens, wte.weight.size(1)).uniform_(-random_range, random_range)
            
    def forward(self, tokens):
        """run forward pass
        Args:
            tokens (torch.long): input tokens before encoding
        Returns:
            torch.float: encoding of text concatenated with learned task specifc embedding
        """
        input_embedding = self.wte(tokens[:, self.n_tokens:])
        learned_embedding = self.learned_embedding.repeat(input_embedding.size(0), 1, 1)
        return torch.cat([learned_embedding, input_embedding], 1)

In [29]:
n_tokens = 20
initialize_from_vocab = True

In [30]:
model = T5ForConditionalGeneration.from_pretrained("t5-base")
tokenizer = T5Tokenizer.from_pretrained("t5-base")
n_tokens = len(WARNING_DL)
s_wte = SoftEmbedding(model.get_input_embeddings(), 
                      n_tokens=n_tokens, 
                      initialize_from_vocab=True)
model.set_input_embeddings(s_wte)
if torch.cuda.is_available():
    model = model.to(device)

/root/miniconda3/lib/python3.8/site-packages/transformers/models/t5/tokenization_t5.py:163: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


In [31]:
model.eval()

T5ForConditionalGeneration(
  (shared): SoftEmbedding(
    (wte): Embedding(32128, 768)
  )
  (encoder): T5Stack(
    (embed_tokens): SoftEmbedding(
      (wte): Embedding(32128, 768)
    )
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=768, out_features=768, bias=False)
              (k): Linear(in_features=768, out_features=768, bias=False)
              (v): Linear(in_features=768, out_features=768, bias=False)
              (o): Linear(in_features=768, out_features=768, bias=False)
              (relative_attention_bias): Embedding(32, 12)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseActDense(
              (wi): Linear(in_features=768, out_features=3072, bias=False)
              (wo): Linear(in_features=

In [32]:
print(s_wte)

SoftEmbedding(
  (wte): Embedding(32128, 768)
)


In [33]:
parameters = list(model.parameters())
for x in parameters[1:]:
    x.requires_grad = False

In [34]:
str_warning_dl_list=[dl_func.__name__ for dl_func in WARNING_DL]
print(str_warning_dl_list)

['out_of_range_low_battery', 'flying_empty_battery', 'empty_battery_water_proof_flying_water', 'flying_not_normal_frame', 'low_battery_low_temperature', 'low_battery_strong_wind', 'low_battery_high_altitude', 'water_proof_precipitation', 'flying_water_low_altitude', 'rain_strong_wind', 'upsidedown_inexperienced', 'extreme_weather_inexperienced', 'gloomy_high_altitude', 'indoor_near_human', 'low_visibility_near_object', 'low_altitude_fast_speed', 'near_fragile_object_near_human', 'near_object_inexperienced', 'near_moving_object', 'reachable_inpath', 'very_close_human', 'very_close_lamp', 'very_close_object']


In [35]:
# Todo: Use dataset instead.

def generate_data(batch_size, n_tokens, related_sensor_data, text_data):

    def yield_data(x_batch, y_batch, l_batch):
            
        x = torch.nn.utils.rnn.pad_sequence(x_batch, batch_first=True)
        y = torch.nn.utils.rnn.pad_sequence(y_batch, batch_first=True)
#         y = torch.cat(y_batch, dim=0)
            
        # Todo: Add prompt mask for different description logic
        m = (x > 0).to(torch.float32)
        decoder_input_ids = torch.full((x.shape[0], y.shape[1]), 1)
        
        if torch.cuda.is_available():
            x = x.to(device)
            y = y.to(device)
            m = m.to(device)
            decoder_input_ids = decoder_input_ids.to(device)
        return x, y, m, decoder_input_ids, l_batch

    x_batch, y_batch, l_batch = [], [], []
    for x, y in zip(related_sensor_data, text_data):
        
        context = x
        inputs = tokenizer(context, return_tensors="pt")
        inputs['input_ids'] = torch.cat([torch.full((1, n_tokens), 1), inputs['input_ids']], 1)
        
        
        outputs=tokenizer(y, return_tensors="pt")
#         print(type(inputs), inputs['input_ids'][0].size(), inputs['input_ids'].size(), 
#               type(outputs), outputs['input_ids'][0].size(), outputs['input_ids'].size())
        
        outputs_ids = torch.cat([torch.full((1, n_tokens - 1), -100), outputs['input_ids']], 1)
        
        x_batch.append(inputs['input_ids'][0])
        y_batch.append(outputs_ids[0])
        l_batch.append(outputs['input_ids'][0])
        if len(x_batch) >= batch_size:
            yield yield_data(x_batch, y_batch, l_batch)
            x_batch, y_batch, l_batch = [], [], []

    if len(x_batch) > 0:
        yield yield_data(x_batch, y_batch, l_batch)
        x_batch, y_batch, l_batch = [], [], []

In [36]:
for x, y, m, dii, true_labels in generate_data(4, n_tokens, related_sensor_data_li, templates_li):
    assert dii.shape == y.shape
    outputs = model(input_ids=x, labels=y, attention_mask=m, decoder_input_ids=dii)
    assert outputs['logits'].shape[:2] == y.shape
    pred_labels = outputs['logits'][:, :, :].argmax(-1).detach().cpu().numpy()
    print("pred_labels:{}".format((pred_labels, pred_labels.shape)))
    break

pred_labels:(array([[32099,     3,     3,     2,     5,     6,     7,     3,     9,
           10,    11,    12,     5,     3,    15,    16,    17,    18,
           19,    20,    21,    22,     3,     1,     2,     2,     2,
            2,     2,     2,     2,     2,     3,     3,     3,     3,
            3,     3,     3,     3,     3,     3,     3,     3,     3,
            3,     3,     3,     3,     3,     3,     3,     3,     3,
            3,     3,     3,     3],
       [    3,     3,     3,     2,     5,     6,     7,     8,     9,
           10,    11,    12,    13,     3,    15,    16,    17,    18,
           19,    20,    21,    22,     3,     2,     2,     2,     2,
            2,     3,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,    41,    41,    41,    41,    41,
           41,    41,    41,    41,     1,     1,     1,     1,     1,
            1,     1,     1,     1],
       [    3,     3,     3,     2,     5,     6,     7,     

In [37]:
batch_size = 32
n_epoch = 50
total_batch = math.ceil(len(related_sensor_data_li) / batch_size)
dev_total_batch = math.ceil(len(templates_li) / batch_size)
use_ce_loss = False
ce_loss = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(s_wte.parameters(), lr=0.5)

for epoch in range(n_epoch):
    print('epoch', epoch)

    all_true_labels = []
    all_pred_labels = []
    losses = []
    pbar = tqdm(enumerate(generate_data(batch_size, n_tokens, train_sensor_data, train_templates)), total=total_batch)
    for i, (x, y, m, dii, true_labels) in pbar:
        all_true_labels += true_labels
        
        optimizer.zero_grad()
        outputs = model(input_ids=x, labels=y, attention_mask=m, decoder_input_ids=dii)
        pred_labels = outputs['logits'][:, :, :].argmax(-1).detach().cpu().numpy().tolist()
        all_pred_labels += pred_labels

        if use_ce_loss:
            logits = outputs['logits'][:, :, :]
            true_labels_tensor = torch.tensor(true_labels, dtype=torch.long).cuda()
            loss = ce_loss(logits, true_labels_tensor)
        else:
            loss = outputs.loss
        loss.backward()
        optimizer.step()
        loss_value = float(loss.detach().cpu().numpy().tolist()) / batch_size
        losses.append(loss_value)

#         acc = accuracy_score(all_true_labels, all_pred_labels)
        pbar.set_description(f'train: loss={np.mean(losses):.4f}')

    all_true_labels = []
    all_pred_labels = []
    losses = []
    with torch.no_grad():
        pbar = tqdm(enumerate(generate_data(batch_size, n_tokens, test_sensor_data, test_templates)), total=dev_total_batch)
        for i, (x, y, m, dii, true_labels) in pbar:
            all_true_labels += true_labels
            outputs = model(input_ids=x, labels=y, attention_mask=m, decoder_input_ids=dii)
            loss = outputs.loss
            loss_value = float(loss.detach().cpu().numpy().tolist()) / batch_size
            losses.append(loss_value)
            pred_labels = outputs['logits'][:, :, :].argmax(-1).detach().cpu().numpy().tolist()
            all_pred_labels += pred_labels
            pbar.set_description(f'dev: loss={np.mean(losses):.4f}')

epoch 0


dev: loss=0.2457:  40%|████      | 21/52 [00:01<00:02, 11.94it/s]


epoch 1


dev: loss=0.1598:  40%|████      | 21/52 [00:01<00:02, 11.97it/s]


epoch 2


train: loss=0.1598:   6%|▌         | 3/52 [00:00<00:08,  5.51it/s]


KeyboardInterrupt: 

In [ ]:
inputs = tokenizer(test_sensor_data[0], return_tensors="pt").to(device)
print(test_sensor_data[0])

# need to pad attention_mask and input_ids to be full seq_len + n_learned_tokens
# even though it does not matter what you pad input_ids with, it's just to make HF happy
inputs['input_ids'] = torch.cat([torch.full((1, n_tokens), 1).to(device), inputs['input_ids']], 1)
inputs['attention_mask'] = torch.cat([torch.full((1,n_tokens), 1).to(device), inputs['attention_mask']], 1)
inputs['decoder_input_ids']=torch.full((1, 100), 1).to(device)
 
outputs = model(**inputs)
print(outputs.logits.shape)
output_ids=outputs.logits[:, n_tokens:, :]
print(output_ids.shape)
print(tokenizer.batch_decode(output_ids.argmax(-1).detach().cpu().numpy().tolist()))
